In [ ]:
!pip install transformers

     |████████████████████████████████| 4.7 MB 8.3 MB/s 
     |████████████████████████████████| 120 kB 78.4 MB/s 
     |████████████████████████████████| 6.6 MB 52.4 MB/s 


In [2]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,BertTokenizer

In [5]:
df = pd.read_csv('/content/sample_data/Sentiment_analysis_data.csv')


In [6]:
df = df.sample(frac=1)
df.tail(10)

,Comment,Tag
91,বিগত ৩০ বছরে লুটপাট কম হলে এতোদিনে মধ্যম আয়ের ...,0
12101,বিএনপি সন্ত্রাসী কর্মকাণ্ডের মাধ্যমে ষড়যন্ত্র ...,0
1371,ভাল উদ্যোগ। স্বাগতম।,1
10102,আর সেই মার্শকেই তার নিজের দেশের মাটিতে ধুলিস্ম...,0
9451,হায়রে মেসি তুমি শুধুই বার্সার,0
3399,শ্রীলংকার খারাপ অবস্থাও বাংলাদেশের স্বাভাবিক অ...,0
12080,আমাদের দেশের মানুষ অনেক অসচেতন।,0
900,পাগলে কিনা কয়আর ছাগলে পাতা চিবায়,0
3805,প্রত্যেক কেন্দ্র কেন্দ্রে ভোটের ফল ঘোষনা করতে ...,1
8156,ব্যাংক ডাকাতদের আইনের অাওতায় আনা বাঞ্চনীয়,1


In [7]:
df['Tag'].value_counts()

1    10269
0     9747
2     2951
Name: Tag, dtype: int64

In [ ]:
tokenizer = BertTokenizer.from_pretrained("csebuetnlp/banglabert_generator")

In [9]:
df['Comment'].iloc[0]

'এদের জন্য ফারমার্স ব্যাংক আদর্শ!'

In [10]:
token = tokenizer.encode_plus(
    df['Comment'].iloc[0],
    max_length = 256,
    truncation=True,
    padding='max_length',
    add_special_tokens=True,
    return_tensor='tf'
)

Keyword arguments {'return_tensor': 'tf'} not recognized.


In [11]:
x_input_ids = np.zeros((len(df),256))
x_attention_mask = np.zeros((len(df),256))


In [12]:
def generate_training_data(df,ids,masks,tokenizer):
  for i, text in tqdm(enumerate(df['Comment'])):
    tokenized_text = tokenizer.encode_plus(
        text,
        max_length=256,
        truncation=True,
        padding = 'max_length',
        add_special_tokens=True,
        return_tensors='tf'
    )
    ids[i, :] = tokenized_text.input_ids
    masks[i,:]= tokenized_text.attention_mask
  return ids,masks

In [13]:
x_input_ids,x_attention_mask=generate_training_data(df,x_input_ids,x_attention_mask,tokenizer)

0it [00:00, ?it/s]

In [14]:
x_input_ids

array([[2.000e+00, 2.824e+03, 9.000e+02, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.000e+00, 1.103e+03, 1.839e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.000e+00, 1.418e+03, 2.111e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [2.000e+00, 4.433e+03, 4.100e+02, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.000e+00, 3.866e+03, 1.947e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.000e+00, 3.182e+03, 7.395e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00]])

In [15]:
labels = np.zeros((len(df),3))

In [16]:
labels[np.arange(len(df)),df['Tag'].values]=1

In [17]:
dataset = tf.data.Dataset.from_tensor_slices((x_input_ids,x_attention_mask,labels))

In [18]:
dataset.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(3,), dtype=tf.float64, name=None))>

In [19]:
def SentimentDatasetMapFunction(input_ids,attention_mask,labels):
  return{
      'input_ids':input_ids,
      'attention_mask':attention_mask
  }, labels

In [20]:
dataset = dataset.map(SentimentDatasetMapFunction)

In [21]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(3,), dtype=tf.float64, name=None))>

In [22]:
dataset = dataset.shuffle(10000).batch(16,drop_remainder=True)

In [23]:
p=14
train_size = int((len(df)//16)*p)

In [24]:
train_size

20090

In [25]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [26]:
from transformers import TFBertModel

In [27]:
bert_Model = TFBertModel.from_pretrained("csebuetnlp/banglabert_generator",from_pt=True)

You are using a model of type electra to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['electra.encoder.layer.6.intermediate.dense.bias', 'electra.encoder.layer.5.attention.self.key.weight', 'electra.encoder.layer.7.attention.output.dense.weight', 'electra.encoder.layer.0.intermediate.dense.bias', 'electra.encoder.layer.4.intermediate.dense.weight', 'electra.embeddings.LayerNorm.bias', 'electra.encoder.layer.0.intermediate.dense.weight', 'electra.encoder.layer.3.attention.output.dense.bias', 'electra.encoder.layer.11.attention.output.dense.weight', 'generator_predictions.LayerNorm.bias', 'electra.encoder.layer.2.intermediate.dense.bias', 'electra.encoder.layer.6.output.LayerNorm.weight', 'electra.encoder.layer.5.attention.output.LayerNorm.bias', 'electra.encoder.layer.7.output.LayerNorm.weight', 'electra.encoder.layer.4.attention.self.key.weight', 'electra.encoder.layer.0.attention.self.value.weight', 'electra.encoder.layer.10.attention.self.query.weight', 'electra.encoder.la

In [28]:
input_ids = tf.keras.layers.Input(shape=(256,),name='input_ids',dtype='int32')
attention_masks = tf.keras.layers.Input(shape=(256,),name='attention_mask',dtype='int32')

bert_embds = bert_Model.bert(input_ids,attention_mask=attention_masks)[1]
intermediate_layer = tf.keras.layers.Dense(512,activation='relu',name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(3,activation='softmax',name='output_layer')(intermediate_layer)

model = tf.keras.Model(inputs=[input_ids,attention_masks],outputs=output_layer)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  17867008    ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [29]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-5,decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [30]:
model.compile(optimizer=optim,loss=loss_func,metrics=[acc])

In [31]:
hist = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=1
)

1435/1435 [==============================] - 265s 178ms/step - loss: 0.8356 - accuracy: 0.6094


In [32]:
model.save('sentiment_model')

In [33]:
loaded_model = tf.keras.models.load_model('sentiment_model')